In [47]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [85]:
from functions import get_candidate_finance_reports_cleaned
from functions import get_candidate_list_cleaned

In [ ]:
def fetch_single_c5(reportId, name, schedule):
    """
    Fetches contributions from single report, parses as dataframe
    """

    post_url = 'https://camptrackext.mt.gov/CampaignTracker/public/viewFinanceReport/prepareDownloadFileFromSearch'
    get_url = 'https://camptrackext.mt.gov/CampaignTracker/public/viewFinanceReport/downloadFile'
    post_payload = {
        'reportId': reportId, # This is from checkbox on candidate report page
        'scheduleCode': schedule, # A is contributions, # B is expenditures
        'fname': name,
    }
    session = requests.Session()
    p = session.post(post_url, post_payload)
    if 'fileName' in p.json():
        r = session.get(get_url, params=p.json())
        if r.text == '':
            print('Empty file. Report ID:', reportId)
            return pd.DataFrame()
        dfi = pd.read_csv(StringIO(r.text), sep='|', error_bad_lines=False, warn_bad_lines=True)
        return dfi
    else:
        print('No file. Report ID:', reportId)
        return pd.DataFrame()

In [86]:
year = '2020'
candidate_type_code = 'SW' # statewide
search_data = {
    'lastName': '',
    'firstName': '',
    'middleInitial': '',
    'electionYear': year,
    'candidateTypeCode': candidate_type_code,
    'officeCode': '',
    'countyCode': '',
    'partyCode': '',
}
all_candidates = get_candidate_list_cleaned(search_data)

In [94]:
# all_candidates

In [95]:
[cand['candidateId'] for cand in all_candidates if (cand['candidateName'] == 'Cooney, Mike  R')]

[16070]

In [82]:
# Functions for fetching summary totals
def fetch_candidate_summaries(candidateId):
    reports = get_candidate_finance_reports_cleaned(candidateId)
    if len(reports) == 0:
        print('Found no reports', candidateId)
    candidateName = reports[0]['candidateName']
    print('Found', len(reports), 'reports for', candidateName)
    report_summaries = []
    for report in reports:
        summary = get_report_summary(report['reportId'], candidateId)
        report_summaries.append(summary)
    return report_summaries
    

def get_report_summary(reportId, candidateId):
    html = fetch_summary_report(reportId, candidateId)
    soup = BeautifulSoup(html.text, 'html.parser')
    parsed = parse_report(soup)
    return parsed

def fetch_summary_report(reportId, candidateId):
    post_url = 'https://camptrackext.mt.gov/CampaignTracker/public/viewFinanceReport/retrieveReport'
    post_payload = {
        'candidateId': candidateId,
        'reportId': reportId,
        'searchPage': 'public'
    }
    session = requests.Session()
    p = session.post(post_url, post_payload)
    return p

def parse_report(soup):
    labels = [
        'previous report',
        'Receipts',
        'Expenditures',
        'Ending Balance',
    ]
    table = soup.find('div', id='summaryAccordionId').find('table')
    return [{label: get_row(table, label)} for label in labels]

def get_row(table, label):
    # label can be partial text or regex
    row = table.find('td', text=re.compile(label)).parent
    # replaces remove "$" and "," from strings
    pri = float(row.find_all('td')[2].text.replace('$','').replace(',','').replace(',',''))
    gen =  float(row.find_all('td')[3].text.replace('$','').replace(',','').replace(',',''))
    return {
        'primary': pri,
        'general': gen,
        'total': round(pri + gen, 2),
    }

In [81]:
get_report_summary('45526', '16017') # 16017 is Schreiner

[{'previous report': {'primary': 0.0, 'general': 0.0, 'total': 0.0}},
 {'Receipts': {'primary': 26375.0, 'general': 6440.0, 'total': 32815.0}},
 {'Expenditures': {'primary': 1411.22, 'general': 0.0, 'total': 1411.22}},
 {'Ending Balance': {'primary': 24963.78,
   'general': 6440.0,
   'total': 31403.78}}]

In [97]:
summaries = fetch_candidate_summaries('16070')

Found 2 reports for Cooney, Mike  R


In [ ]:
# def checkSummaries

In [98]:
summaries

[[{'previous report': {'primary': 120581.92,
    'general': 21635.0,
    'total': 142216.92}},
  {'Receipts': {'primary': 185618.43, 'general': 15388.0, 'total': 201006.43}},
  {'Expenditures': {'primary': 164724.95,
    'general': 30.0,
    'total': 164754.95}},
  {'Ending Balance': {'primary': 141475.4,
    'general': 36993.0,
    'total': 178468.4}}],
 [{'previous report': {'primary': 0.0, 'general': 0.0, 'total': 0.0}},
  {'Receipts': {'primary': 228228.68, 'general': 21795.0, 'total': 250023.68}},
  {'Expenditures': {'primary': 107646.76,
    'general': 160.0,
    'total': 107806.76}},
  {'Ending Balance': {'primary': 120581.92,
    'general': 21635.0,
    'total': 142216.92}}]]

In [103]:
summaries[0][1]['Receipts']['total'] + summaries[0][1]['Receipts']['total']

402012.86